In [1]:
import os
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
import time

Using TensorFlow backend.


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [3]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [4]:
def create_model(input_shape, anchors, num_classes, load_pretrained=False, freeze_body=False,
            weights_path='model_data/darknet53_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [5]:
def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=False, freeze_body=False,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [6]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    index_map = dict(zip(class_names, range(num_classes)))
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

In [7]:
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [8]:
annotation_path = 'train.txt'
log_dir = 'weights/'
classes_path = 'model_data/voc_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'

class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
input_shape = (416,416) # multiple of 32, hw

model = create_model(input_shape, anchors, len(class_names), load_pretrained=True, freeze_body=2 )

Create YOLOv3 model with 9 anchors and 1 classes.
Load weights model_data/darknet53_weights.h5.
Freeze the first 249 layers of total 252 layers.


In [9]:
logging = TensorBoard(log_dir='./logs')
"""logging = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=1,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=False, # 是否可视化梯度直方图
                 write_images=False,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)"""
checkpoint = ModelCheckpoint(log_dir + "ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5",
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)


batch_size = 64 # 64到512 （2的平方）
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.shuffle(lines)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

Train on 459 samples, val on 51 samples, with batch size 64.


In [10]:
# 除了分类层，其他层冻结后进行训练
model.compile(optimizer=Adam(lr=1e-2), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.summary()

Train on 459 samples, val on 51 samples, with batch size 64.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
________________________________________________

In [11]:
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint, early_stopping])

#model.save_weights( 'weights/trained_weights_stage_1_%s.h5' % time.strftime("%Y-%m-%d", time.localtime()))

Epoch 1/50
7/7 [==============================] - 894s 128s/step - loss: 2974.9071 - val_loss: 4530.0010
Epoch 2/50
7/7 [==============================] - 834s 119s/step - loss: 206.5542 - val_loss: 3728.1125
Epoch 3/50
7/7 [==============================] - 804s 115s/step - loss: 134.0218 - val_loss: 3322.4255
Epoch 4/50
7/7 [==============================] - 804s 115s/step - loss: 116.5816 - val_loss: 2853.1951
Epoch 5/50
7/7 [==============================] - 825s 118s/step - loss: 106.6342 - val_loss: 2494.6135
Epoch 6/50
7/7 [==============================] - 811s 116s/step - loss: 101.7608 - val_loss: 2181.7429
Epoch 7/50
7/7 [==============================] - 793s 113s/step - loss: 98.4157 - val_loss: 1884.9563
Epoch 8/50
7/7 [==============================] - 799s 114s/step - loss: 92.0469 - val_loss: 1620.3628
Epoch 9/50
7/7 [==============================] - 766s 109s/step - loss: 84.4469 - val_loss: 1374.6460
Epoch 10/50
7/7 [==============================] - 804s 115s/step 

In [12]:
model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=100,
                initial_epoch=50,
                callbacks=[logging, checkpoint, early_stopping])

Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 51/100
7/7 [==============================] - 821s 117s/step - loss: 49.8458 - val_loss: 51.0056
Epoch 52/100
7/7 [==============================] - 827s 118s/step - loss: 48.5197 - val_loss: 48.6845
Epoch 53/100
7/7 [==============================] - 815s 116s/step - loss: 45.8518 - val_loss: 44.1789
Epoch 54/100
7/7 [==============================] - 788s 113s/step - loss: 45.7191 - val_loss: 45.3178
Epoch 55/100
7/7 [==============================] - 844s 121s/step - loss: 43.6548 - val_loss: 42.2692
Epoch 56/100
7/7 [==============================] - 796s 114s/step - loss: 45.4324 - val_loss: 42.4175
Epoch 57/100
7/7 [==============================] - 757s 108s/step - loss: 44.2348 - val_loss: 44.4429
Epoch 58/100
7/7 [==============================] - 798s 114s/step - loss: 44.7180 - val_loss: 43.8330
Epoch 59/100
7/7 [==============================] - 809s 116s/step - loss: 44.2174 - val_loss: 41.7882
Epoch 60/100

In [13]:
model.compile(optimizer=Adam(lr=1e-4), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=100,
                initial_epoch=50,
                callbacks=[logging, checkpoint, early_stopping])

Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 51/100
7/7 [==============================] - 761s 109s/step - loss: 38.7783 - val_loss: 40.1239
Epoch 52/100
7/7 [==============================] - 771s 110s/step - loss: 38.7811 - val_loss: 38.5500
Epoch 53/100
7/7 [==============================] - 776s 111s/step - loss: 38.5767 - val_loss: 38.1123
Epoch 54/100
7/7 [==============================] - 797s 114s/step - loss: 38.2381 - val_loss: 38.5404
Epoch 55/100
7/7 [==============================] - 767s 110s/step - loss: 38.4802 - val_loss: 39.1478
Epoch 56/100
7/7 [==============================] - 765s 109s/step - loss: 39.2428 - val_loss: 38.7432
Epoch 57/100
7/7 [==============================] - 764s 109s/step - loss: 37.8883 - val_loss: 40.7499
Epoch 58/100
7/7 [==============================] - 761s 109s/step - loss: 38.9560 - val_loss: 37.0112
Epoch 59/100
7/7 [==============================] - 775s 111s/step - loss: 39.0958 - val_loss: 39.1381
Epoch 60/100

In [14]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
# 解冻后继续训练
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
#model.summary()

#batch_size = 20 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=1000,
                initial_epoch=100,
                callbacks=[logging, checkpoint])

model.save_weights( 'weights/trained_weights_stage_2_%s.h5' % time.strftime("%Y-%m-%d", time.localtime()))

Unfreeze all of the layers.
Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 101/1000
7/7 [==============================] - 2965s 424s/step - loss: 34.4873 - val_loss: 73.5999
Epoch 102/1000
7/7 [==============================] - 2916s 417s/step - loss: 30.9525 - val_loss: 168.8791
Epoch 103/1000
7/7 [==============================] - 2917s 417s/step - loss: 29.0578 - val_loss: 142.3768
Epoch 104/1000
7/7 [==============================] - 2932s 419s/step - loss: 25.9824 - val_loss: 90.7807
Epoch 105/1000
7/7 [==============================] - 2929s 418s/step - loss: 24.3094 - val_loss: 64.0105
Epoch 106/1000
7/7 [==============================] - 2933s 419s/step - loss: 23.6496 - val_loss: 70.2890
Epoch 107/1000
7/7 [==============================] - 2947s 421s/step - loss: 22.2075 - val_loss: 92.1957
Epoch 108/1000
7/7 [==============================] - 2952s 422s/step - loss: 22.0178 - val_loss: 100.9131
Epoch 109/1000
7/7 [==============================] - 2950s 

KeyboardInterrupt: 

In [15]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
# 解冻后继续训练
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-5), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
#model.summary()

#batch_size = 20 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=1000,
                initial_epoch=100,
                callbacks=[logging, checkpoint])

model.save_weights( 'weights/trained_weights_stage_2_%s.h5' % time.strftime("%Y-%m-%d", time.localtime()))

Unfreeze all of the layers.
Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 101/1000
7/7 [==============================] - 2957s 422s/step - loss: 15.7430 - val_loss: 16.3594
Epoch 102/1000
7/7 [==============================] - 2962s 423s/step - loss: 15.1912 - val_loss: 15.8572
Epoch 103/1000
7/7 [==============================] - 2963s 423s/step - loss: 15.6012 - val_loss: 15.4068
Epoch 104/1000
7/7 [==============================] - 2999s 428s/step - loss: 14.9255 - val_loss: 16.4462
Epoch 105/1000
7/7 [==============================] - 2975s 425s/step - loss: 15.1896 - val_loss: 16.5807
Epoch 107/1000
7/7 [==============================] - 2970s 424s/step - loss: 15.1309 - val_loss: 16.7646
Epoch 108/1000
7/7 [==============================] - 2990s 427s/step - loss: 14.9748 - val_loss: 15.0427
Epoch 109/1000
7/7 [==============================] - 2964s 423s/step - loss: 15.0688 - val_loss: 16.7606
Epoch 110/1000
7/7 [==============================] - 2975s 425

KeyboardInterrupt: 

In [16]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
# 解冻后继续训练
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-6), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
#model.summary()

#batch_size = 20 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=1000,
                initial_epoch=100,
                callbacks=[logging, checkpoint])

model.save_weights( 'weights/trained_weights_stage_2_%s.h5' % time.strftime("%Y-%m-%d", time.localtime()))

Unfreeze all of the layers.
Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 101/1000
7/7 [==============================] - 3017s 431s/step - loss: 15.1933 - val_loss: 16.5733
Epoch 102/1000
7/7 [==============================] - 2947s 421s/step - loss: 14.6722 - val_loss: 15.8451
Epoch 103/1000
7/7 [==============================] - 2993s 428s/step - loss: 14.6101 - val_loss: 15.3595
Epoch 104/1000
7/7 [==============================] - 2994s 428s/step - loss: 14.6436 - val_loss: 15.4782
Epoch 105/1000
7/7 [==============================] - 3013s 430s/step - loss: 14.7798 - val_loss: 14.2827
Epoch 106/1000
7/7 [==============================] - 2979s 426s/step - loss: 14.6030 - val_loss: 15.4845
Epoch 107/1000
7/7 [==============================] - 2968s 424s/step - loss: 14.4972 - val_loss: 15.9618
Epoch 108/1000
7/7 [==============================] - 2974s 425s/step - loss: 14.6319 - val_loss: 16.5377
Epoch 109/1000
7/7 [==============================] - 2961s 423

KeyboardInterrupt: 

In [18]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
# 解冻后继续训练
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
#model.summary()

#batch_size = 20 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=1000,
                initial_epoch=100,
                callbacks=[logging, checkpoint])

model.save_weights( 'weights/trained_weights_stage_2_%s.h5' % time.strftime("%Y-%m-%d", time.localtime()))

Unfreeze all of the layers.
Train on 459 samples, val on 51 samples, with batch size 64.
Epoch 101/1000
1/7 [===>..........................] - ETA: 44:13 - loss: nan

KeyboardInterrupt: 

In [ ]:
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
model.summary()

batch_size = 10 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=100,
            initial_epoch=50,
            callbacks=[logging, checkpoint, reduce_lr])

Unfreeze all of the layers.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_________________________________________________________________________________

__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, None, None, 2 196608      concatenate_1[0][0]              
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, None, None, 2 1024        conv2d_61[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_60 (LeakyReLU)      (None, None, None, 2 0           batch_normalization_60[0][0]     
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, None, None, 5 1179648     leaky_re_lu_60[0][0]             
__________________________________________________________________________________________________
batch_normalization_61 (BatchNo (None, None, None, 5 2048        conv2d_62[0][0]                  
__________

Epoch 51/100
45/45 [==============================] - 2861s 64s/step - loss: 49.3558 - val_loss: 49.1816
Epoch 52/100
45/45 [==============================] - 2814s 63s/step - loss: 47.1507 - val_loss: 50.2089
Epoch 53/100
45/45 [==============================] - 2833s 63s/step - loss: 45.6691 - val_loss: 47.9862
Epoch 54/100
45/45 [==============================] - 2821s 63s/step - loss: 44.3587 - val_loss: 47.6187
Epoch 55/100
45/45 [==============================] - 2829s 63s/step - loss: 41.1704 - val_loss: 45.3588
Epoch 56/100
45/45 [==============================] - 2825s 63s/step - loss: 37.0978 - val_loss: 43.9791
Epoch 57/100
45/45 [==============================] - 2820s 63s/step - loss: 36.0630 - val_loss: 41.4664
Epoch 58/100
45/45 [==============================] - 2823s 63s/step - loss: 33.3954 - val_loss: 40.6201
Epoch 59/100
45/45 [==============================] - 2826s 63s/step - loss: 32.2907 - val_loss: 39.6228
Epoch 60/100
45/45 [==============================] - 2

In [ ]:
model.save_weights(log_dir + 'trained_weights_final.h5')

In [15]:
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')
model.summary()

batch_size = 10 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=100,
            initial_epoch=0,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
model.save_weights(log_dir + 'trained_weights_final_2.h5')

Unfreeze all of the layers.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_________________________________________________________________________________

__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, None, None, 2 131072      leaky_re_lu_63[0][0]             
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, None, None, 2 1024        conv2d_65[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_64 (LeakyReLU)      (None, None, None, 2 0           batch_normalization_64[0][0]     
__________________________________________________________________________________________________
conv2d_68 (Conv2D)              (None, None, None, 1 32768       leaky_re_lu_64[0][0]             
__________________________________________________________________________________________________
batch_normalization_66 (BatchNo (None, None, None, 1 512         conv2d_68[0][0]                  
__________

Epoch 1/100
45/45 [==============================] - 2889s 64s/step - loss: 23.6497 - val_loss: 24.5845
Epoch 2/100
45/45 [==============================] - 2819s 63s/step - loss: 23.7998 - val_loss: 24.9540
Epoch 3/100
45/45 [==============================] - 2847s 63s/step - loss: 23.2088 - val_loss: 25.5619
Epoch 4/100
45/45 [==============================] - 2850s 63s/step - loss: 23.2740 - val_loss: 23.0187
Epoch 5/100
45/45 [==============================] - 2844s 63s/step - loss: 22.7558 - val_loss: 23.3320
Epoch 6/100
45/45 [==============================] - 2821s 63s/step - loss: 22.4335 - val_loss: 22.2286
Epoch 7/100
45/45 [==============================] - 2806s 62s/step - loss: 22.5202 - val_loss: 24.0535
Epoch 8/100
45/45 [==============================] - 2803s 62s/step - loss: 22.8720 - val_loss: 23.7524
Epoch 9/100
45/45 [==============================] - 2804s 62s/step - loss: 22.3174 - val_loss: 22.7908
Epoch 10/100
45/45 [==============================] - 2825s 63s/

In [9]:
def train_yolov3(img_root_path):
    data_dir = img_root_path
    root_dir_path = data_dir
    annotation_train = os.path.join(data_dir, 'ImageSets', 'Main', 'train.txt')
    annotation_val = os.path.join(data_dir, 'ImageSets', 'Main', 'val.txt')
    log_dir = 'logs/000/'
    classes_path = 'model_data/handwriting_chinese_classes.txt'
    anchors_path = 'model_data/yolo_anchors.txt'
    
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    index_map = dict(zip(class_names, range(num_classes)))
    anchors = get_anchors(anchors_path)
    
    input_shape = (416,416) # multiple of 32, hw
    
    is_tiny_version = len(anchors)==6 # default setting
    
    if is_tiny_version:
        model = create_tiny_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
    else:
        model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/yolo_weights.h5') # make sure you know what you freeze
    
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
    
    
    #划分训练集和验证集
    #val_split = 0.1
    with open(annotation_train) as train_f:
        train_lines = train_f.readlines()
    with open(annotation_val) as val_f:
        val_lines = val_f.readlines()
        
    np.random.seed(10101)
    np.random.shuffle(train_lines)
    np.random.shuffle(val_lines)
    
    num_train = len(train_lines)
    num_val = len(val_lines)
    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 8
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.summary()
        model.fit_generator(data_generator_wrapper(root_dir_path, train_lines, batch_size, input_shape, anchors, num_classes, class_names),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(root_dir_path, val_lines, batch_size, input_shape, anchors, num_classes, class_names),
                validation_steps=max(1, num_val//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint])
        model.save_weights(log_dir + 'trained_weights_stage_1.h5')
    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')
        model.summary()

        batch_size = 8 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(root_dir_path, train_lines, batch_size, input_shape, anchors, num_classes, class_names),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(root_dir_path, val_lines, batch_size, input_shape, anchors, num_classes, class_names),
            validation_steps=max(1, num_val//batch_size),
            epochs=100,
            initial_epoch=50,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        model.save_weights(log_dir + 'trained_weights_final.h5')

In [ ]:
train_yolov3(img_root_path)

Create YOLOv3 model with 9 anchors and 1 classes.
Train on 510 samples, val on 100 samples, with batch size 8.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]     

Epoch 1/50
63/63 [==============================] - 5146s 82s/step - loss: 1131.4719 - val_loss: 6464.6820
Epoch 2/50
63/63 [==============================] - 5030s 80s/step - loss: 97.3187 - val_loss: 202.2798
Epoch 3/50
63/63 [==============================] - 4928s 78s/step - loss: 53.5494 - val_loss: 71.6535
Epoch 4/50
63/63 [==============================] - 4846s 77s/step - loss: 37.4725 - val_loss: 43.1836
Epoch 5/50
63/63 [==============================] - 4968s 79s/step - loss: 29.6198 - val_loss: 31.0900
Epoch 6/50
63/63 [==============================] - 4894s 78s/step - loss: 24.8079 - val_loss: 25.4702
Epoch 7/50
63/63 [==============================] - 4838s 77s/step - loss: 21.7529 - val_loss: 21.9398
Epoch 8/50
63/63 [==============================] - 4826s 77s/step - loss: 19.6773 - val_loss: 52.3972
Epoch 9/50
63/63 [==============================] - 4917s 78s/step - loss: 18.2389 - val_loss: 538.0900
Epoch 10/50
63/63 [==============================] - 4822s 77s/step